In [64]:
import itertools
import json
import os

import h5py
import matplotlib.pyplot as plt
import meshio
import numpy as np

from adios2 import FileReader, Stream
from mpi4py import MPI

import commons, geometry, utils

In [52]:
markers = commons.Markers()

In [3]:
def fetch_simulation_metadata(workdir, Wa_n=1e-3, Wa_p=1e-3, gamma=25.0):
    metadata_file = os.path.join(workdir, f"{str(float(Wa_n))}-{str(float(Wa_p))}", f"{str(float(gamma))}", "simulation.json")
    with open(metadata_file) as f:
        data = json.load(f)#[0]
    return data

def read_node_ids_for_marker(h5_file_obj, marker):
    line_ids = np.where(np.asarray(h5_file_obj['data2']) == marker)[0]
    lines = np.asarray(h5_file_obj['data1'])[line_ids, :]
    nodes = set(list(itertools.chain.from_iterable(lines[line_ids, :].reshape(-1, 1).tolist())))
    return nodes

In [4]:
I_left = []
I_mid = []
I_right = []
I_insulated = []
positive_wa = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
# norm_stdev_i_pos_am = []
ssb_planar_workdir = "output/ssb_planar/150-40-0/20-55-20/5.0e-06/"
for Wa_p in positive_wa:
    data = fetch_simulation_metadata(ssb_planar_workdir, Wa_p=Wa_p, gamma=10)
    I_left.append(float(data["Current at negative am - electrolyte boundary"].strip(" A")))
    I_mid.append(float(data["Current at electrolyte - positive am boundary"].strip(" A")))
    I_right.append(float(data["Current at right boundary"].strip(" A")))
    I_insulated.append(float(data["Current at insulated boundary"].strip(" A")))
    # norm_stdev_i_pos_am.append(float(data["stdev i positive charge transfer (normalized)"]))

fig, ax = plt.subplots()
ax.semilogx(positive_wa, I_left, 'r+', label='Left')
ax.semilogx(positive_wa, I_mid, 'bo', label='Middle')
ax.semilogx(positive_wa, I_right, 'k-', label='Right')
ax.set_ylabel('current [A]')
# ax.set_ylim([0, 0.025])
ax.set_xlabel('Wa')
ax.legend()
plt.tight_layout()
plt.savefig('figures/sipdg/planar/planar-current-interfaces.png')

fig, ax = plt.subplots()
ax.semilogx(positive_wa, I_insulated, 'r', label='Left')
ax.set_ylabel('current [A]')
ax.set_xlabel('Wa')
# ax.legend()
plt.tight_layout()
plt.savefig('figures/sipdg/planar/planar-current-insulated.png')

FileNotFoundError: [Errno 2] No such file or directory: 'output/ssb_planar/150-40-0/20-55-20/5.0e-07/0.001-0.001/10.0/simulation.json'

In [ ]:
I_left = []
I_mid = []
I_right = []
I_insulated = []
positive_wa = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
norm_stdev_i_pos_am = []
ssb_unstructured_workdir = "output/ssb_unstructured/150-40-0/20-55-20/5.0e-07/"

for Wa_p in positive_wa:
    data = fetch_simulation_metadata(ssb_unstructured_workdir, Wa_p=Wa_p, gamma=10)
    I_left.append(float(data["Current at negative am - electrolyte boundary"].strip(" A")))
    I_mid.append(float(data["Current at electrolyte - positive am boundary"].strip(" A")))
    I_right.append(float(data["Current at right boundary"].strip(" A")))
    I_insulated.append(float(data["Current at insulated boundary"].strip(" A")))
    norm_stdev_i_pos_am.append(float(data["stdev i positive charge transfer (normalized)"]))

fig, ax = plt.subplots()
ax.semilogx(positive_wa, I_left, 'r+', label='Left')
ax.semilogx(positive_wa, I_mid, 'bo', label='Middle')
ax.semilogx(positive_wa, I_right, 'k-', label='Right')
ax.set_ylabel('current [A]')
# ax.set_ylim([0, 0.025])
ax.set_xlabel('Wa')
ax.legend()
plt.tight_layout()
plt.savefig('figures/sipdg/complex/complex-current-interfaces.png')


fig, ax = plt.subplots()
ax.semilogx(positive_wa, I_insulated, 'r', label='Left')
ax.set_ylabel('I [A]')
ax.set_xlabel('Wa')
# ax.legend()
plt.tight_layout()
plt.savefig('figures/sipdg/complex/complex-current-insulated.png')

fig, ax = plt.subplots()
ax.semilogx(positive_wa, norm_stdev_i_pos_am, 'r')
ax.set_ylabel(r'$\frac{\sigma_{i_n}}{\mu_{i_n}}$', fontsize='xx-large')
ax.set_xlabel('Wa')
# ax.legend()
plt.tight_layout()
plt.savefig('figures/sipdg/complex/complex-normstdev-i-pos-am.png')

In [30]:
comm = MPI.COMM_WORLD
data_file = "output/ssb_unstructured/150-40-0/20-55-20/5.0e-07/0.001-1000.0-0.01/10.0/potential.bp"
with FileReader(data_file) as s:
    vars = s.available_variables()
    for name, info in vars.items():
        print("variable_name: " + name, end=" ")
        for key, value in info.items():
            print("\t" + key + ": " + value, end="\n")
        
    geom = s.read('geometry')
    nodes = s.read('connectivity')
    data = s.read('potential')
    print(geom.shape)
    print(np.unique(nodes[:, 0]), np.unique(nodes))
    print(s.read('types'))

variable_name: NumberOfEntities 	AvailableStepsCount: 1
	Max: 15852
	Min: 15344
	Shape: 8
	SingleValue: false
	Type: uint32_t
variable_name: NumberOfNodes 	AvailableStepsCount: 1
	Max: 47556
	Min: 46032
	Shape: 8
	SingleValue: false
	Type: uint32_t
variable_name: connectivity 	AvailableStepsCount: 1
	Max: 47555
	Min: 0
	Shape: 
	SingleValue: false
	Type: int64_t
variable_name: geometry 	AvailableStepsCount: 1
	Max: 0.00015
	Min: -5.04939e-23
	Shape: 
	SingleValue: false
	Type: double
variable_name: potential 	AvailableStepsCount: 1
	Max: 1
	Min: -6.72615e-07
	Shape: 
	SingleValue: false
	Type: double
variable_name: step 	AvailableStepsCount: 1
	Max: 0
	Min: 0
	Shape: 
	SingleValue: true
	Type: double
variable_name: types 	AvailableStepsCount: 1
	Max: 69
	Min: 69
	Shape: 
	SingleValue: true
	Type: uint32_t
variable_name: vtkGhostType 	AvailableStepsCount: 1
	Max: 1
	Min: 0
	Shape: 
	SingleValue: false
	Type: uint8_t
variable_name: vtkOriginalPointIds 	AvailableStepsCount: 1
	Max: 371336

In [33]:
h5_file = "output/ssb_unstructured/150-40-0/20-55-20/5.0e-07/0.001-0.001-0.1/10.0/current.h5"
f = h5py.File(h5_file, 'r')
print(list(f.keys()))

['Function', 'Mesh']


In [79]:
mesh = f['Mesh']
arr = np.asarray(mesh['mesh']['topology'])

In [39]:
geom = mesh['mesh']['geometry']

In [81]:
arr[:5, :]

array([[0, 1, 2],
       [2, 1, 3],
       [2, 3, 4],
       [5, 1, 0],
       [3, 1, 6]])

In [43]:
msh = meshio.read("output/ssb_unstructured/150-40-0/20-55-20/5.0e-07/mesh.msh")
line_mesh = geometry.create_mesh(msh, "line")
line_mesh.write("output/ssb_unstructured/150-40-0/20-55-20/5.0e-07/lines.xdmf")

In [44]:
f_geo = h5py.File("output/ssb_unstructured/150-40-0/20-55-20/5.0e-07/lines.h5", 'r')
f_geo.keys()

<KeysViewHDF5 ['data0', 'data1', 'data2']>

In [71]:
coords = np.asarray(f_geo['data0'])

In [72]:
coords[:5, :]

array([[1.5e-04, 0.0e+00, 0.0e+00],
       [1.5e-04, 4.0e-05, 0.0e+00],
       [3.0e-05, 0.0e+00, 0.0e+00],
       [3.0e-05, 9.0e-06, 0.0e+00],
       [2.9e-05, 1.0e-05, 0.0e+00]])

In [73]:
geom[:5, :]

array([[1.49699351e-04, 4.00000000e-05, 0.00000000e+00],
       [1.49749348e-04, 3.99421553e-05, 0.00000000e+00],
       [1.49799344e-04, 4.00000000e-05, 0.00000000e+00],
       [1.49849341e-04, 3.99134033e-05, 0.00000000e+00],
       [1.49899338e-04, 4.00000000e-05, 0.00000000e+00]])